In [ ]:
# %% Prepare necessary COSVF default inputs
import os,sys
import json
import numpy as np
import pandas as pd
import time
from ast import literal_eval
import matplotlib.pyplot as plt
from pandas.core.arrays.categorical import contains

# %% Load pyVIN
calvin_dir = os.path.abspath('../')
if str(calvin_dir)!=sys.path:
    sys.path.append(calvin_dir)
from calvin import CALVIN, cosvfea
from calvin import *


In [ ]:
scenario_climate = '1995HTD'
scenario_name = 'wsip{}csii'.format(scenario_climate)
lf_dir = '../my-models/calvin-lf'

# %% PF scenarios
pf_dir = '../my-models/calvin-pf'


# %% calsim flows
cs_flows = pd.read_csv('csii-wsip-flows-and-storages.csv')
cs_sr_init = pd.read_csv('csii-wsip-init-storages.csv')
cs_map = pd.read_csv('csii-calvin-mapping.csv')
cs_map['SV'] = cs_map['SV'].apply(literal_eval)
cs_map['DV'] = cs_map['DV'].apply(literal_eval)
cs_map['S'] = cs_map['S'].apply(literal_eval)
#  convert cfs to TAF/month
cs_flows['C300'] =  cs_flows['C300']*60.37/1000
cs_flows.iloc[:,3:] = cs_flows.iloc[:,3:].round(3)


##### LOAD ORIGINAL PF LINKS ####
#  %% load perfect foresight links
pf_links = pd.read_csv(os.path.join('../my-models/calvin-pf-original', 'links82yr.csv'))
# remove debug nodes
pf_links = pf_links.loc[
    (~pf_links.i.str.contains('DBUG')) & (~pf_links.j.str.contains('DBUG'))]
# create a column for edges (w/o dates)
pf_links.insert(0, 'edge', 
    value=pf_links.i.str.split('.').str[0]+'_'+pf_links.j.str.split('.').str[0])
pf_links.insert(1,'i_node',
    value=pf_links.i.map(lambda x: x.split('.')[0]))
pf_links.insert(2,'j_node',
    value=pf_links.j.map(lambda x: x.split('.')[0]))
pf_links.insert(0, 'year',
    value=pd.DatetimeIndex(pf_links.i.str.split('.').str[1]).year)
pf_links.insert(1, 'month',
    value=pd.DatetimeIndex(pf_links.i.str.split('.').str[1]).month)
pf_links.insert(0, 'date',
    value=pd.DatetimeIndex(pf_links.i.str.split('.').str[1]))
# %%
pf_links_mod = pf_links.copy()



#### INITIAL STOAGES FOR PF ####
# %% set initial storages on PF links
for sr in cs_sr_init.sr:
    pf_links_mod.loc[pf_links_mod.edge=='INITIAL_{}'.format(sr),['lower_bound','upper_bound']] = cs_sr_init.loc[cs_sr_init.sr==sr,'calsim'].iloc[0]
    print(pf_links_mod.loc[pf_links_mod.edge=='INITIAL_{}'.format(sr),['lower_bound','upper_bound']])



#### INFLOWS FOR PF ####
# %% historical flows
pf_flows_compare = pd.DataFrame()
for idx,row in cs_map.iterrows():
    if row.SV or row.DV:
        link = str(row.Inflow_link)
        print(link)
        flows_original = pf_links.loc[pf_links.edge==link].copy()
        print(flows_original.lower_bound.sum())
        if row.SV:
            flows_csii = cs_flows.loc[cs_flows.scenario==scenario_climate,row.SV].sum(axis=1)
        else:
            flows_csii = cs_flows.loc[cs_flows.scenario==scenario_climate,row.DV].sum(axis=1)
        print(flows_csii.sum())
        pf_links_mod.loc[pf_links_mod.edge==link,'lower_bound'] = flows_csii.values
        pf_links_mod.loc[pf_links_mod.edge==link,'upper_bound'] = flows_csii.values
        pf_flows_compare = pf_flows_compare.append(pd.DataFrame({
            'date':flows_original.date,
            'year':flows_original.year,
            'month':flows_original.month,
            'link':link,
            'calsim':flows_csii,
            'calvin':flows_original.lower_bound
        }))
# %% 
pf_links_mod_out = pf_links_mod[['i','j','k','cost','amplitude','lower_bound','upper_bound']]
pf_links_mod_out.to_csv(os.path.join(pf_dir,'links82yr.csv'.format(scenario_name)),index=False)



#### INFLOWS FOR LF ###
# %% Extract monthly inflows to csv
# load inflows 
inflow_qwry = pf_links_mod.loc[(pf_links_mod.i.str.contains('INFLOW'))]
# split j to node and date
inflows = inflow_qwry['j'].str.split('.',expand=True)
inflows.columns = ['j','date']
inflows['date'] = pd.DatetimeIndex(inflows['date'])
inflows.set_index('date',inplace=True)
# get inflow values
inflows.insert(1,'flow_taf', value = inflow_qwry['lower_bound'].values)

# %%  save out inflows output
inflows.to_csv(os.path.join(lf_dir,'inflows.csv'))



### INITIAL STORAGE ON LF LINKS ###
# %% Init storages 
links = pd.read_csv('links.csv')
# %% set initial storages on LF links
for sr in cs_sr_init.sr:
    links.loc[(links.i=='INITIAL') & (links.j.str.contains(sr)), ['lower_bound','upper_bound']] = cs_sr_init.loc[cs_sr_init.sr==sr,'calsim'].iloc[0]
# %%
links.to_csv(os.path.join(lf_dir,'links.csv'),index=False)


### REVISE R_DICT ####
# %%
with open('r-dict.json') as f: 
    R_DICT = json.load(f)
for sr in cs_sr_init.sr:
    R_DICT[sr]['eop_init'] = float(cs_sr_init.loc[cs_sr_init.sr==sr,'calsim'].iloc[0])
# %% save out the reservoir dictionary to json file
R_DICT_out = R_DICT
with open(os.path.join(lf_dir,'r-dict.json'), 'w') as json_file:
    json.dump(R_DICT_out, json_file, 
        sort_keys=False, indent=4, separators=(',', ': '))


#################################
#################################
#################################





### FIXED STORAGE ON LF LINKS ###
# %% set scenario name to fixed storage (FS)
scenario_name = 'fs'.format(scenario_climate)

In [ ]:
eoy_storages = pd.DataFrame()
for idx,row in cs_map.iterrows():
    if row.S:
        link = str(row.SR)
        print(link)
        eoy_storage = cs_flows.loc[cs_flows.scenario==scenario_climate, ['datesim',row.S[0]]][11::12]
        eoy_storage.columns = ['date','lower_bound']
        eoy_storage.insert(2,'upper_bound',value=eoy_storage.lower_bound)
        eoy_storage.insert(1,'i',value='{}.'.format(link)+eoy_storage['date'].astype(str))
        eoy_storage.insert(2,'j',value='FINAL')
        eoy_storage.insert(3,'k',value=0)
        eoy_storages = eoy_storages.append(eoy_storage)
# %% load variable constraints and append final storage contraints
variable_constraints = pd.read_csv('variable-constraints.csv')
variable_constraints = variable_constraints.append(eoy_storages)
# %%
lf_dir = '../../../my-models/calvin-lf-{}'.format(scenario_name)
variable_constraints.to_csv(os.path.join(lf_dir,'variable-constraints.csv'),index=False)
links.to_csv(os.path.join(lf_dir,'links.csv'),index=False)
inflows.to_csv(os.path.join(lf_dir,'inflows.csv'))


### REVISE RESERVOIR DICTIONARY ###
# %% Reservoirs identified as COSVF canditates (Type 1)
r_list = pf_links_mod.loc[(pf_links_mod.i_node.str.startswith('INITIAL'))].j_node.unique()
r_type1 = ['SR_BER','SR_BUL','SR_CLK_INV','SR_CMN',
            'SR_EBMUD','SR_HTH','SR_ISB','SR_LL_ENR','SR_LVQ',
            'SR_PAR','SR_PNF','SR_RLL_CMB',
            'SR_SFAGG','SR_GNT']
r_type2 = ['GW_01', 'GW_02', 'GW_03','GW_04', 'GW_05', 'GW_06','GW_07',
           'GW_08', 'GW_09', 'GW_10', 'GW_11', 'GW_12', 'GW_13', 'GW_14', 'GW_15',
           'GW_16', 'GW_17', 'GW_18', 'GW_19', 'GW_20', 'GW_21',
           'GW_AV', 'GW_CH', 'GW_EW', 'GW_IM', 'GW_MJ', 'GW_MWD',
           'GW_OW', 'GW_SBV', 'GW_SC', 'GW_SD', 'GW_VC']
# reservoir dictionary for calvin limited foresight run
r_dict = dict()
i = 0
for r in r_list:
    # initial storage value
    initial_storage = pf_links_mod.loc[
        (pf_links_mod.i_node=='INITIAL') & (pf_links_mod.j_node==r)].lower_bound
    # lower bound on carryover
    lb_9 = pf_links_mod.loc[
        (pf_links_mod.i_node==r) & (pf_links_mod.j_node==r) & 
        (pf_links_mod.k==0) & (pf_links_mod.month==9)].lower_bound.min()
    # upper bound on carryover from first year
    ub_9 = pf_links_mod.loc[
        (pf_links_mod.i_node==r) & (pf_links_mod.j_node==r) & 
        (pf_links_mod.month==9) & (pf_links_mod.year==1922)].upper_bound.sum()
    # check COSVF Type 1 to index COSVF param
    if r in r_type1:
        r_type, cosvf_param_index, k_count, i = 1, [i,i+1], 15, i+2
    elif r in r_type2:
        r_type, cosvf_param_index, k_count, i = 2, i, 2, i+1
    else:
        r_type, cosvf_param_index, k_count = 0, None, 1
    # add to reservoir dictionary
    r_dict[r] = dict([
        ('eop_init',initial_storage.values[0]),
        ('lb',lb_9),
        ('ub',ub_9),
        ('type',r_type), 
        ('cosvf_param_index',cosvf_param_index),
        ('k_count',k_count)])
# %% save out the reservoir dictionary to json file
lf_dir = '../../../my-models/calvin-lf-{}'.format(scenario_name)
with open(os.path.join(lf_dir,'r-dict.json'), 'w') as json_file:
    json.dump(r_dict, json_file, 
        sort_keys=False, indent=4, separators=(',', ': '))


###  REVISE COSVF BASED ON RTYPES IN USE FOR FIXED STORAGE ###
# %% Create default COSVF params
param=['pmin','pmax']
rtype1_list = list({key: value for key, value in r_dict.items() if value['type'] == 1}.keys())
rtype2_list = list({key: value for key, value in r_dict.items() if value['type'] == 2}.keys())
pos_r_list = rtype2_list + list(np.repeat(rtype1_list, len(param)))
cosvf_pminmax = pd.DataFrame({'value':
    list(np.repeat([-1e-6], len(rtype2_list))) + list(np.tile([-2e-2,-2e-2], len(rtype1_list)))})
cosvf_pminmax.insert(0,'r',value=pos_r_list)
cosvf_pminmax.insert(1,'param',value=list(['p'] * len(rtype2_list) + param * len(rtype1_list)))
# %% save out default COSVF params
lf_dir = '../../../my-models/calvin-lf-{}'.format(scenario_name)
cosvf_pminmax.to_csv(os.path.join(lf_dir,'cosvf-params-.csv'),index=False)

